In [2]:
# version 1 
import pymysql
import paramiko
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import sshtunnel as sshtunnel
import logging
import configparser
import os 
import datetime
import pickle
import json




date_time = datetime.datetime.now()

LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s" 
logging.basicConfig( filename='logger.log' ,  level= logging.INFO , format= LOG_FORMAT  )
logger = logging.getLogger()


#Change this to the private key in local 
mypkey = paramiko.Ed25519Key.from_private_key_file('/Users/icmec/.ssh/wpengine_icmec27012023')


parser = configparser.ConfigParser()
current_directory =  os.getcwd()
parser.read(f"{current_directory}/conf/db.conf")

sql_hostname = parser.get("db_config", "sql_hostname") 
sql_username =  parser.get("db_config", "sql_username") 
sql_password = parser.get("db_config", "sql_password")
sql_main_database = parser.get("db_config", "sql_main_database")
sql_port = parser.getint("db_config", "sql_port")
ssh_host = parser.get("db_config", "ssh_host")
ssh_user = parser.get("db_config", "ssh_user")
ssh_port = parser.getint("db_config", "ssh_port")

global resultJson; resultJson = {};
#resultJson['Date'] = date_time.strftime("%d/%m/%Y")

def getNumberOfMembers(curr):
        curr.execute("SELECT COUNT(DISTINCT(ID)) FROM `wp_users` ")
        output = curr.fetchone()
        print(output[0])
        resultJson['numMembers'] = output[0]

def getVersion(curr):
        curr.execute("SELECT VERSION() ")
        output = curr.fetchone()
        print(output[0]) 

def getNumberOfMessage(curr):
        curr.execute("SELECT COUNT(*) FROM `wp_bp_messages_messages` ")
        output = curr.fetchone()
        print(output[0]) 
        resultJson['numMessages'] = output[0]

def getNumberOfDocs(curr):
        curr.execute("SELECT COUNT( DISTINCT(document_id) ) FROM `wp_bp_document_meta` ")
        output = curr.fetchone()
        print(output[0]) 
        resultJson['numDocs'] = output[0]

def getNumberOfDocs(curr):
        curr.execute("SELECT COUNT( DISTINCT(document_id) ) FROM `wp_bp_document_meta` ")
        output = curr.fetchone()
        print(output[0]) 
        resultJson['numDocs'] = output[0]

def getNumberOfConnections(curr):
        curr.execute("SELECT COUNT(*) FROM `wp_bp_friends`")
        output = curr.fetchone()
        print(output[0]) 
        resultJson['numConnections'] = output[0]

def getNumberOfInvitations(curr):
        curr.execute("SELECT COUNT(*) FROM `wp_bp_invitations`")
        output = curr.fetchone()
        print(output[0]) 
        resultJson['numInvitations'] = output[0]

def getPosts(curr):
        curr.execute("SELECT COUNT(*) AS Posts,SUM(comment_count) as Total_Comments FROM `wp_posts` WHERE post_type = 'post';")
        output = curr.fetchone()
        print(output) 
        resultJson['Posts'] = { 'numPosts': output[0] , 'numComments': int(output[1])  }
        #resultJson['numInvitations'] = output[0]

def getEvents(curr):
        curr.execute("SELECT COUNT(*) AS Posts,SUM(comment_count) as Total_Comments FROM `wp_posts` WHERE post_type = 'tribe_events';")
        output = curr.fetchone()
        print(output) 
        resultJson['Events'] = { 'numEvents': output[0] , 'numComments': int(output[1])  }
        #resultJson['numInvitations'] = output[0]

#save the result
def Save():
    name =  date_time.strftime("%d%m%Y")
    with open('data/'+name+'.json', 'w') as fp:
        json.dump(resultJson, fp)





try:
        with SSHTunnelForwarder( (ssh_host,ssh_port),ssh_username=ssh_user,ssh_pkey=mypkey,remote_bind_address=(sql_hostname, sql_port) ) as tunnel:
                
                #establish  connection to the member portal via pymysql
                conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                passwd=sql_password, db=sql_main_database,
                port=tunnel.local_bind_port)
                cur = conn.cursor()
                #query = '''SELECT VERSION();'''
                #data = pd.read_sql_query(query, conn)
               
                #Run querires on Member portal db 
                getNumberOfMembers(cur)
                getVersion(cur)
                getNumberOfMessage(cur)
                getNumberOfDocs(cur)
                getNumberOfConnections(cur)
                getNumberOfInvitations(cur)
                getPosts(cur)
                getEvents(cur)
               
                conn.close()
                logger.info('Successfully extracted information')
                print(resultJson)
except Exception as e:
        logger.error(e)

Save()

KeyboardInterrupt: 

In [3]:
import pymysql
import paramiko
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import sshtunnel as sshtunnel
import logging
import configparser
import os 
import datetime
import pickle
import json

In [2]:
mypkey = paramiko.Ed25519Key.from_private_key_file('/Users/icmec/.ssh/wpengine_icmec27012023')


parser = configparser.ConfigParser()
current_directory =  os.getcwd()
parser.read(f"{current_directory}/conf/db.conf")

sql_hostname = parser.get("db_config", "sql_hostname") 
sql_username =  parser.get("db_config", "sql_username") 
sql_password = parser.get("db_config", "sql_password")
sql_main_database = parser.get("db_config", "sql_main_database")
sql_port = parser.getint("db_config", "sql_port")
ssh_host = parser.get("db_config", "ssh_host")
ssh_user = parser.get("db_config", "ssh_user")
ssh_port = parser.getint("db_config", "ssh_port")

In [13]:
global data;
global resultJson; resultJson = {} 

def getNumberOfMembers(con):
   query = '''SELECT CAST(`user_registered` AS DATE) AS date ,COUNT(*) AS cnt FROM `wp_users` WHERE user_login != 'wpengine' GROUP BY CAST(`user_registered` AS DATE)'''
   membersDf =  pd.read_sql_query(query, con)
   membersDf['cumulative'] = membersDf['cnt'].cumsum()
   membersDf = membersDf.set_index('date')
   resultJson['numMembers'] = membersDf.to_dict()

def getNumberOfMessage(con):
   query = '''SELECT CAST(`date_sent` AS DATE) AS date,COUNT(*) as cnt FROM `wp_bp_messages_messages` GROUP BY CAST(`date_sent` AS DATE)'''
   messagesDf =  pd.read_sql_query(query, con)
   messagesDf['cumulative'] = messagesDf['cnt'].cumsum()
   messagesDf = messagesDf.set_index('date')
   resultJson['numMessages'] = messagesDf.to_dict()

def getNumberOfInvitations(con):
   query = '''SELECT CAST(`date_modified` AS DATE) AS date,COUNT(*) as cnt FROM `wp_bp_invitations` GROUP BY CAST(`date_modified` AS DATE)'''
   invitationsDf =  pd.read_sql_query(query, con)
   invitationsDf['cumulative'] = invitationsDf['cnt'].cumsum()
   invitationsDf = invitationsDf.set_index('date')
   resultJson['numInvitations'] = invitationsDf.to_dict()

def getNumberOfConnections(con):
   query = '''SELECT CAST(`date_created` AS DATE) AS date,COUNT(*) as cnt FROM `wp_bp_friends` GROUP BY CAST(`date_created` AS DATE)'''
   connectionsDf =  pd.read_sql_query(query, con)
   connectionsDf['cumulative'] = connectionsDf['cnt'].cumsum()
   connectionsDf = connectionsDf.set_index('date')
   resultJson['numConnections'] = connectionsDf.to_dict()

def getPosts(con):
   query = '''SELECT CAST(`post_date` AS DATE) AS date,COUNT(*) as post,SUM(comment_count) as comments  FROM `wp_posts` where post_type = 'tribe_events' GROUP BY CAST(`post_date` AS DATE)'''
   postsDf =  pd.read_sql_query(query, con)
   postsDf['cumulative'] = postsDf['post'].cumsum()
   postsDf = postsDf.set_index('date')
   resultJson['Posts'] = postsDf.to_dict()

def getEvents(con):
   query = '''SELECT CAST(`post_date` AS DATE) AS date,COUNT(*) as event,SUM(comment_count) as comments  FROM `wp_posts` where post_type = 'tribe_events' GROUP BY CAST(`post_date` AS DATE)'''
   eventsDf =  pd.read_sql_query(query, con)
   eventsDf['cumulative'] = eventsDf['event'].cumsum()
   eventsDf = eventsDf.set_index('date')
   resultJson['Events'] = eventsDf.to_dict()

def getProfileUpdaters(con):
   query = '''SELECT CAST(wp_bp_activity.date_recorded AS DATE)  AS date , wp_users.display_name as name,wp_bp_activity.type FROM `wp_bp_activity`  LEFT JOIN `wp_users` on wp_bp_activity.user_id = wp_users.ID WHERE type = \'updated_profile\''''
   UpdatersDf =  pd.read_sql_query(query, con)
   return UpdatersDf
   #eventsDf['cumulative'] = eventsDf['event'].cumsum()
   UpdatersDf = UpdatersDf.set_index('date')
   resultJson['Updaters'] = UpdatersDf.to_dict()
   print(resultJson['Updaters'])

    

In [21]:
try:
        with SSHTunnelForwarder( (ssh_host,ssh_port),ssh_username=ssh_user,ssh_pkey=mypkey,remote_bind_address=(sql_hostname, sql_port) ) as tunnel:
                
                #establish  connection to the member portal via pymysql
                conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                passwd=sql_password, db=sql_main_database,
                port=tunnel.local_bind_port)
                cur = conn.cursor()
                data = getProfileUpdaters(conn)
                # getNumberOfMembers(conn)
                # getNumberOfMessage(conn)
                # getNumberOfInvitations(conn)
                # getNumberOfConnections(conn)
                # getPosts(conn)
                # getEvents(conn)
                # query = '''SELECT CAST(`user_registered` AS DATE) AS date ,COUNT(*) AS cnt FROM `wp_users` WHERE user_login != 'wpengine' GROUP BY CAST(`user_registered` AS DATE)'''
                # data = pd.read_sql_query(query, conn)
                # display(data)
                #Run querires on Member portal db 
                # getNumberOfMembers(cur)
                # getVersion(cur)
                # getNumberOfMessage(cur)
                # getNumberOfDocs(cur)
                # getNumberOfConnections(cur)
                # getNumberOfInvitations(cur)
                # getPosts(cur)
                # getEvents(cur)
               
                conn.close()
                #logger.info('Successfully extracted information')
                #print(resultJson)
except Exception as e:
        print('ds')
        #logger.error(e)



In [23]:
#data = data.set_index('date')
data.to_dict( orient='records' )
#pd.DataFrame.from_dict( data.to_dict( orient='records' ))

# resultJson['Updaters'] = data.to_dict()
# print(resultJson['Updaters'])

[{'date': datetime.date(2022, 7, 18),
  'name': 'Unknown Member',
  'type': 'updated_profile'},
 {'date': datetime.date(2022, 7, 19),
  'name': 'Russell Banks',
  'type': 'updated_profile'}]

In [140]:
pd.DataFrame.from_dict( resultJson['numConnections'])

,cnt,cumulative
2022-07-20,2,2
2022-07-28,1,3
2022-08-23,1,4
2022-08-25,1,5
2022-08-30,1,6
2022-09-04,1,7
2022-09-13,2,9
2022-10-07,1,10
2022-10-21,3,13
2022-10-24,1,14


In [79]:
date_time = datetime.datetime.now()

def Save():
    name =  date_time.strftime("%d%m%Y")
    with open('data/sample'+name+'.json', 'w') as fp:
        json.dump(resultJson, fp)

In [99]:
def pickleSave():
    name =  date_time.strftime("%d%m%Y")
    with open('data/'+ name + '.pickle', 'wb') as handle:
        pickle.dump(resultJson, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
def pickleLoad():
    with open('data/portalData.pickle', 'rb') as handle:
        obj  = pickle.load(handle)
    return obj

obj = pickleLoad()

In [25]:
obj.keys()

dict_keys(['numMembers', 'numMessages', 'numInvitations', 'numConnections', 'Posts', 'Events', 'Updaters'])

In [10]:
pd.DataFrame.from_dict(obj)


,Updaters
name,"{2022-07-18: 'Unknown Member', 2022-07-19: 'Ru..."
type,"{2022-07-18: 'updated_profile', 2022-07-19: 'u..."


In [12]:
obj = pickleLoad()


In [16]:
obj.keys()

dict_keys(['numMembers', 'numMessages', 'numInvitations', 'numConnections', 'Posts', 'Events'])

In [133]:
pd.DataFrame.from_dict(obj['numMembers'])

,cnt,cumulative
2022-07-15,1,1
2022-07-18,5,6
2022-07-19,2,8
2022-09-02,1,9
2022-09-21,1,10
2022-10-04,3,13
2022-10-05,2,15
2022-10-06,1,16
2022-10-07,1,17
2022-10-14,1,18


In [109]:
pickleLoad()


In [96]:
global b
with open('data/filename.pickle', 'rb') as handle:
    b = pickle.load(handle)


In [98]:
b['numMembers']

{'cnt': {datetime.date(2022, 7, 15): 1,
  datetime.date(2022, 7, 18): 5,
  datetime.date(2022, 7, 19): 2,
  datetime.date(2022, 9, 2): 1,
  datetime.date(2022, 9, 21): 1,
  datetime.date(2022, 10, 4): 3,
  datetime.date(2022, 10, 5): 2,
  datetime.date(2022, 10, 6): 1,
  datetime.date(2022, 10, 7): 1,
  datetime.date(2022, 10, 14): 1,
  datetime.date(2022, 10, 20): 2,
  datetime.date(2022, 10, 21): 2,
  datetime.date(2022, 10, 23): 2,
  datetime.date(2022, 10, 24): 5,
  datetime.date(2022, 10, 28): 1,
  datetime.date(2022, 11, 24): 1,
  datetime.date(2023, 1, 19): 1},
 'cumulative': {datetime.date(2022, 7, 15): 1,
  datetime.date(2022, 7, 18): 6,
  datetime.date(2022, 7, 19): 8,
  datetime.date(2022, 9, 2): 9,
  datetime.date(2022, 9, 21): 10,
  datetime.date(2022, 10, 4): 13,
  datetime.date(2022, 10, 5): 15,
  datetime.date(2022, 10, 6): 16,
  datetime.date(2022, 10, 7): 17,
  datetime.date(2022, 10, 14): 18,
  datetime.date(2022, 10, 20): 20,
  datetime.date(2022, 10, 21): 22,
  dat

In [80]:
Save()

TypeError: keys must be str, int, float, bool or None, not date

In [81]:
test = pd.DataFrame.from_dict(resultJson['Events'])

In [ ]:
pd.read_json('Path/to/File.json')

In [93]:
test.to_json()

'{"level_0":{"0":0,"1":1,"2":2,"3":3},"index":{"0":1660003200000,"1":1661731200000,"2":1663718400000,"3":1664150400000},"event":{"0":1,"1":2,"2":4,"3":1},"comments":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},"cumulative":{"0":1,"1":3,"2":7,"3":8}}'

In [89]:
test.reset_index(inplace=True)
sam  = test.(orient='list' )

In [90]:
test.from_dict(sam)

,level_0,index,event,comments,cumulative
0,0,2022-08-09,1,0.0,1
1,1,2022-08-29,2,0.0,3
2,2,2022-09-21,4,0.0,7
3,3,2022-09-26,1,0.0,8


In [84]:
pd.DataFrame.from_dict( test.to_json(orient='records') , orient= 'records' ) 

ValueError: Expected 'index', 'columns' or 'tight' for orient parameter. Got 'records' instead

In [60]:
#pd.DataFrame.from_dict(resultJson['numMembers']) 

,cnt,cumulative
2022-07-15,1,1
2022-07-18,5,6
2022-07-19,2,8
2022-09-02,1,9
2022-09-21,1,10
2022-10-04,3,13
2022-10-05,2,15
2022-10-06,1,16
2022-10-07,1,17
2022-10-14,1,18
